# Data Wrangling
## Objectives
- Prepare 2 clean and well-formatted csv files for upload to BigQuery and use in Data Viz
  1. GHG historical emissions table
  2. Socioeconomics indicator table

## 1. Data wrangling tasks for Historical Emissions table
- [x] Drop **Source** column as they all have the same value "CAIT"
- [x] Trim all the string values
- [x] Delete EUU and WORLD in Country
- [x] Unpivot all the years columns -> make the table from wide to long form
- [x] Restructure Sector columns
  - [x] Delete "Total..." lines 
  - [x] Create Main Sector columns including "Non-energy" and "Energy"
  - [x] Reclassify *Energy* lines in **Sector** columns
    - Energy unspecified = Energy - All sub-energy sectors (If we have data)
- [x] Deal with missing data? -> Missing = No Data and do nothing!
- [x] Create processed file in relevant path.

In [31]:
# import libraries
import os
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
# import GHG emissions data
emissions_df = pd.read_csv("../data/raw/CW_HistoricalEmissions_CAIT.csv")

In [3]:
# first look on the data
emissions_df.head()

,Country,Source,Sector,Gas,1990,1991,1992,1993,1994,1995,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,AFG,CAIT,Total excluding LUCF,All GHG,11.968960,12.198490,11.416530,11.498150,11.535340,11.969910,...,27.115900,29.413060,28.800070,27.015200,26.764020,27.086070,26.894000,26.526930,27.689580,28.636500
1,AFG,CAIT,Total including LUCF,All GHG,9.580450,9.809976,9.028016,9.109638,9.146831,9.581401,...,27.237800,29.166840,28.553850,26.768980,26.517800,26.839850,27.048660,26.681590,27.844240,28.791160
2,AFG,CAIT,Energy,All GHG,2.614785,2.430223,1.544761,1.457698,1.366936,1.277973,...,8.265836,10.289940,9.627950,7.730256,6.831263,8.087269,7.421360,6.944151,8.228242,8.267433
3,AFG,CAIT,Industrial Processes,All GHG,0.052709,0.055380,0.058051,0.061942,0.064613,0.067284,...,0.249415,0.314576,0.394037,0.465779,0.523990,0.601661,0.789337,0.910724,1.075560,1.209746
4,AFG,CAIT,Agriculture,All GHG,8.072853,8.396465,8.409491,8.486480,8.523959,8.957016,...,15.788380,15.902260,15.777790,15.724860,16.220450,15.114810,15.317920,15.223620,14.854280,15.544770


In [4]:
emissions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10903 entries, 0 to 10902
Data columns (total 34 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Country  10903 non-null  object 
 1   Source   10903 non-null  object 
 2   Sector   10903 non-null  object 
 3   Gas      10903 non-null  object 
 4   1990     10639 non-null  float64
 5   1991     10693 non-null  float64
 6   1992     10745 non-null  float64
 7   1993     10757 non-null  float64
 8   1994     10759 non-null  float64
 9   1995     10782 non-null  float64
 10  1996     10784 non-null  float64
 11  1997     10785 non-null  float64
 12  1998     10785 non-null  float64
 13  1999     10785 non-null  float64
 14  2000     10834 non-null  float64
 15  2001     10833 non-null  float64
 16  2002     10835 non-null  float64
 17  2003     10835 non-null  float64
 18  2004     10835 non-null  float64
 19  2005     10861 non-null  float64
 20  2006     10861 non-null  float64
 21  2007     108

In [5]:
print(emissions_df["Source"].value_counts())

# delete column Source as all of its rows have only one value "CAIT"
emissions_df.drop(columns="Source", inplace=True)

CAIT    10903
Name: Source, dtype: int64


In [6]:
# trim values in string columns
emissions_df["Country"] = emissions_df["Country"].str.strip()
emissions_df["Sector"] = emissions_df["Sector"].str.strip()
emissions_df["Gas"] = emissions_df["Gas"].str.strip()

In [7]:
# delete EU and WORLD lines
emissions_df = emissions_df[(emissions_df['Country'] != 'EUU') & (emissions_df['Country'] != 'WORLD')]

In [8]:
emissions_df['Country'].value_counts().sort_values()

LIE    27
AND    32
RWA    32
COK    49
NIU    49
       ..
MOZ    57
YEM    57
ARG    57
PER    57
CUB    57
Name: Country, Length: 193, dtype: int64

In [9]:
# create mising table with total mising values and proportion of missing values
def create_missing_table(df):
    missing_data = df.isna().sum().sort_values(ascending=False)
    missing_data_percent = (df.isna().sum().sort_values(ascending=False)/len(df.index)).round(3)

    missing_data_table = pd.concat([missing_data, missing_data_percent],axis=1)
    missing_data_table.columns = ['Total missing values', 'Proportion of missing values']
    return missing_data_table

create_missing_table(emissions_df)

,Total missing values,Proportion of missing values
1990,264,0.024
1991,210,0.019
1992,158,0.015
1993,146,0.014
1994,144,0.013
1995,121,0.011
1996,119,0.011
1997,118,0.011
1998,118,0.011
1999,118,0.011


In [10]:
# unpivot all the year columns to transform emissions_df from wide to long format 
id_vars_tuple = ("Country", "Gas", "Sector")
value_vars_array = np.arange(1990, 2020, dtype=int).astype(str)
emissions_df = pd.melt(emissions_df, id_vars=id_vars_tuple, value_vars=value_vars_array, var_name="Year", value_name="GHG Emissions")

In [11]:
# drop all "Total..." sectors
emissions_df = emissions_df[(emissions_df['Sector'] != 'Total excluding LUCF') & (emissions_df['Sector'] != 'Total including LUCF')]

In [12]:
# create Agg Sector column with "Energy" and "Non-energy" value
sectors = emissions_df['Sector'].unique().tolist()
agg_sectors = ['Energy', 'Non-energy', 'Non-energy', 'Non-energy', 'Non-energy', 'Non-energy',
               'Energy', 'Energy', 'Energy', 'Energy', 'Energy', 'Energy']
map_agg_sectors = dict(zip(sectors, agg_sectors))

emissions_df['Agg Sector'] = emissions_df['Sector'].map(map_agg_sectors)
emissions_df.insert(2, 'Agg Sector', emissions_df.pop('Agg Sector'))

In [13]:
emissions_df[emissions_df['GHG Emissions'].isna()].groupby('Country').count().sort_values(by='Gas')

,Gas,Agg Sector,Sector,Year,GHG Emissions
Country,,,,,
NZL,1,1,1,1,0
BTN,1,1,1,1,0
CMR,1,1,1,1,0
ARM,2,2,2,2,0
TJK,2,2,2,2,0
...,...,...,...,...,...
KHM,120,120,120,120,0
SUR,240,240,240,240,0
LAO,242,242,242,242,0


In [14]:
# Calculate the GHG Emissions of Energy Unspecified sector
# Energy Unspecified is the difference between Energy and sum of all Energy sub-sectors

sum_energy = emissions_df.groupby(['Country', 'Gas', 'Year', 'Agg Sector'])['GHG Emissions'].sum().reset_index()
# sum_energy = GHG emissions of Energy + sum(GHG emissions of all Energy sub-sector)
sum_energy = sum_energy[sum_energy['Agg Sector'] != 'Non-energy']
sum_energy.rename(columns={'Agg Sector':'Sector', 'GHG Emissions':'Sum Emissions'}, inplace=True)

emissions_df = emissions_df.merge(sum_energy, how='left', on=['Country', 'Gas', 'Year', 'Sector'])
# As Sum Emissions = Energy Emissions + sum(all Energy sub-sector emissions) (aka Sub-Energies Emissions)
# => Sub-Energies Emissions = Sum Emissions - Energy Emissions
emissions_df['Sub-Energies Emissions'] = np.where(emissions_df['Sum Emissions'].notna(), 
                                                  emissions_df['Sum Emissions'] - emissions_df['GHG Emissions'], 
                                                  np.nan)
# Energy unspecified Emissions = Energy Emissions - Sub-Energies Emissions
emissions_df['GHG Emissions'] = np.where(emissions_df['Sub-Energies Emissions'].isna(),
                                         emissions_df['GHG Emissions'],
                                         np.where(emissions_df['GHG Emissions'] - emissions_df['Sub-Energies Emissions'] > 0, 
                                                  emissions_df['GHG Emissions'] - emissions_df['Sub-Energies Emissions'], 
                                                  0)
                                        )

# drop redundant columns and change Energy value in Sector column to Energy Unspecified
emissions_df.drop(columns=['Sum Emissions', 'Sub-Energies Emissions'], inplace=True)
emissions_df['Sector'] = np.where(emissions_df['Sector'] == 'Energy', 'Energy Unspecified', emissions_df['Sector'])

In [15]:
emissions_df = emissions_df.astype({'Country':'category', 'Gas':'category', 'Agg Sector':'category', 'Sector':'category', 'Year':'int32'})

In [16]:
emissions_df.to_csv("../data/processed/historical_ghg_emissions_processed.csv", index=False)

In [44]:
emissions_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 265890 entries, 0 to 265889
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype   
---  ------         --------------   -----   
 0   Country        265890 non-null  category
 1   Gas            265890 non-null  category
 2   Agg Sector     265890 non-null  category
 3   Sector         265890 non-null  category
 4   Year           265890 non-null  int32   
 5   GHG Emissions  263719 non-null  float64 
dtypes: category(4), float64(1), int32(1)
memory usage: 6.3 MB


## 2. Data wrangling tasks for Socioeconomic Table
- [x] Drop unnecessary columns: 'Unnamed: 0', 'Indicator Code', '1960' -> '1989'
- [x] Melt them to unpivot year columns, wide to long table
- [x] Delete World and EU lines and change Korea Dem. People Rep. name
- [x] Merge them using Country Name and Country Code
- [x] Change data type
- [x] Save file to data/processed folder

In [18]:
gdp_df = pd.read_csv("../data/raw/CW_gdp.csv")
pop_df = pd.read_csv("../data/raw/CW_population.csv")

In [19]:
gdp_df.head()

,Unnamed: 0,Country Name,Country Code,Indicator Code,1960,1961,1962,1963,1964,1965,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,0,Afghanistan,AFG,NY.GDP.MKTP.CD,537777811.1,548888895.6,546666677.8,751111191.1,800000044.4,1.006667e+09,...,1.585657e+10,1.780429e+10,2.000160e+10,2.056107e+10,2.048489e+10,1.990711e+10,1.801775e+10,1.886995e+10,1.835388e+10,1.929110e+10
1,1,Angola,AGO,NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,8.379950e+10,1.117900e+11,1.280530e+11,1.367100e+11,1.457120e+11,1.161940e+11,1.011240e+11,1.221240e+11,1.013530e+11,8.881570e+10
2,2,Albania,ALB,NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.192693e+10,1.289077e+10,1.231983e+10,1.277622e+10,1.322814e+10,1.138685e+10,1.186120e+10,1.301969e+10,1.514702e+10,1.527918e+10
3,3,Andorra,AND,NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,3.449967e+09,3.629204e+09,3.188809e+09,3.193704e+09,3.271808e+09,2.789870e+09,2.896679e+09,3.000181e+09,3.218316e+09,3.154058e+09
4,4,United Arab Emirates,ARE,NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,2.897870e+11,3.506660e+11,3.745910e+11,3.901080e+11,4.031370e+11,3.581350e+11,3.570450e+11,3.856060e+11,4.222150e+11,4.211420e+11


In [20]:
gdp_df['Indicator Code'].unique()[0]

'NY.GDP.MKTP.CD'

In [21]:
pop_df.head()

,Unnamed: 0,Country Name,Country Code,Indicator Code,1960,1961,1962,1963,1964,1965,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,0,Afghanistan,AFG,SP.POP.TOTL,8996973.0,9169410.0,9351441.0,9543205.0,9744781.0,9956320.0,...,29185507.0,30117413.0,31161376.0,32269589.0,33370794.0,34413603.0,35383128.0,36296400.0,37172386.0,38041754.0
1,1,Angola,AGO,SP.POP.TOTL,5454933.0,5531472.0,5608539.0,5679458.0,5735044.0,5770570.0,...,23356246.0,24220661.0,25107931.0,26015780.0,26941779.0,27884381.0,28842484.0,29816748.0,30809762.0,31825295.0
2,2,Albania,ALB,SP.POP.TOTL,1608800.0,1659800.0,1711319.0,1762621.0,1814135.0,1864791.0,...,2913021.0,2905195.0,2900401.0,2895092.0,2889104.0,2880703.0,2876101.0,2873457.0,2866376.0,2854191.0
3,3,Andorra,AND,SP.POP.TOTL,13411.0,14375.0,15370.0,16412.0,17469.0,18549.0,...,84449.0,83747.0,82427.0,80774.0,79213.0,78011.0,77297.0,77001.0,77006.0,77142.0
4,4,United Arab Emirates,ARE,SP.POP.TOTL,92418.0,100796.0,112118.0,125130.0,138039.0,149857.0,...,8549988.0,8946777.0,9141596.0,9197910.0,9214175.0,9262900.0,9360980.0,9487203.0,9630959.0,9770529.0


In [22]:
pop_df['Indicator Code'].unique()[0]

'SP.POP.TOTL'

In [23]:
# Since the structure of these 2 table is really similar we can create a function to do the wrangling for both
def process_socioecon_data(df):
    # drop 'Unnamed: 0' and 'Indicator code' columns
    df.drop(columns=['Unnamed: 0'], inplace=True)
    ## get indicator to identify it is gdp_df or pop_df that we are working on
    indicator = df['Indicator Code'].unique()[0]
    df.drop(columns=['Indicator Code'], inplace=True)

    # drop 1960 -> 1989 year columns
    df.drop(df.columns[[x for x in range(2,2+1989-1960+1)]], axis=1, inplace=True)

    # melt 1990 -> 2019 year columns to turn table from wide to long form
    id_vars_tuple = ('Country Name', 'Country Code')
    value_vars_array = np.arange(1990, 2020, dtype=int).astype(str)
    df = pd.melt(df, id_vars=id_vars_tuple, 
                    value_vars=value_vars_array, 
                    var_name='Year', 
                    value_name='GDP' if indicator == 'NY.GDP.MKTP.CD' else 'Population')

    df = df[(df['Country Code'] != 'EUU') & (df['Country Code'] != 'WORLD')]

    df.loc[df['Country Code'] == 'PRK', 'Country Name'] = 'Korea, Dem. People Rep.'

    return df

gdp_df = process_socioecon_data(gdp_df)
pop_df = process_socioecon_data(pop_df)

In [26]:
# merge the 2 datasets together
socioecon_df = gdp_df.merge(pop_df, how='inner', on=['Country Name', 'Country Code', 'Year'])

In [46]:
socioecon_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5880 entries, 0 to 5879
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   Country Name  5880 non-null   category
 1   Country Code  5880 non-null   category
 2   Year          5880 non-null   int32   
 3   GDP           5552 non-null   float64 
 4   Population    5809 non-null   float64 
dtypes: category(2), float64(2), int32(1)
memory usage: 195.0 KB


In [42]:
# change data type
socioecon_df = socioecon_df.astype({'Country Name':'category', 'Country Code':'category', 'Year':'int32'})

In [45]:
socioecon_df.to_csv("../data/processed/socioeconomics_processed.csv", index=False)